In [1]:
import torch
import numpy as np
from easydict import EasyDict as edict

In [2]:
cfg = edict()
cfg.INIT = edict()
cfg.DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

cfg.SCALES = (480, 576, 688, 864, 1200)
cfg.MAX_SIDE = 2000
cfg.K = 3
cfg.It = 0.5
cfg.NMSIoU = 0.3
cfg.INIT.MEAN = 0
cfg.INIT.DEVI = 0.01
cfg.INIT.BAIS = 0
cfg.BATCH_SIZE = 2
cfg.WD = 5e-4
cfg.LR = 1e-3

cfg.LR_07STEP = 12
cfg.LR_07_MUL = 0.1
cfg.EPOCH_07 = 28




SAVE_PATH = "../models/"
LOG_PATH = "../logs/"
OUT_PATH = "../outs/"

In [4]:
def draw_box(img, boxes):
    """
    img : PIL Image
    boxes: np.darray shape (N, 4)
    """
    p = np.asarray(img)
    for box in boxes:
        cv2.rectangle(p, (box[0], box[1]), (box[2], box[3]), (255, 255, 0))
    plt.imshow(p)
    
def get_model_name(propose_way, year, model_name):
    name = ""
    if propose_way == "selective_search":
        name += "ssw_"
    else:
        name += "eb_"
    
    name += str(year)+ "_" 
    name += model_name

    return name

def one2allbox_iou(target_box, others):
    """
     calculate the iou of box A to list of boxes
     target_box : Tensor()  1 * 4 
     others : Tensor()      N * 4 
     return  N * 1  ...iou
    
    """

    # get the min of xmax and ymax which organize the Intersection
    max_xy = torch.min(target_box[:, 2:], others[:, 2:]) 
    min_xy = torch.max(target_box[:, :2], others[:, :2])
    # get the xdistance and y distance
    # add 1 because distance = point2 - point1 + 1
    inter_wh = torch.clamp((max_xy - min_xy + 1), min=0)
    I = inter_wh[:, 0] * inter_wh[:, 1]
    A = (target_box[:, 2] - target_box[:, 0] + 1) * (target_box[:, 3] - target_box[:, 1] + 1)
    B = (others[:, 2] - others[:, 0] + 1) * (others[:, 3] - others[:, 1] + 1)
    return I / (A + B - I)


def write_log(path, content):
    with open(path, 'a') as f:
        f.write(content + "\n")